In [1]:
import lightgbm as lgb
import ember
import sys
import os 
if os.path.abspath('../../') not in sys.path:
    sys.path.append(os.path.abspath('../../'))
from utils import utils
import json
import pickle
from etl.lief_feature import PEFeatureExtractor

### **Training**

In [22]:
import pprint
with open('./lightgbm_config.json', "r") as f:
    json_data = f.read()
    dict_data = json.loads(json_data)

pprint.pprint(dict_data)


{'bagging_fraction': 0.9,
 'bagging_freq': 1,
 'bagging_seed': 0,
 'boost_from_average': True,
 'boosting': 'gbdt',
 'device_type': 'cpu',
 'early_stopping_rounds': 10,
 'feature_fraction': 0.9,
 'feature_fraction_bynode': 0.9,
 'feature_fraction_seed': 0,
 'first_metric_only': True,
 'is_unbalance': True,
 'lambda_l1': 0,
 'lambda_l2': 1.0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': -1,
 'metric': ['binary_logloss', 'auc', 'binary_error'],
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'num_iterations': 500,
 'num_leaves': 64,
 'num_threads': 0,
 'objective': 'binary',
 'seed': 0,
 'sigmoid': 1.0,
 'task': 'train',
 'tree_learner': 'serial',
 'verbosity': 2}


### **Using Ember**

In [ ]:
ember.create_vectorized_features("/data/enber2018")
ember.create_metadata("/data/ember2018")


In [ ]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features("/data/ember2018/")
metadata_dataframe = ember.read_metadata("/data/ember2018/")

In [ ]:
lgbm_model = ember.train_model("/data/ember2018/")
lgbm_model = lgb.Booster(model_file="/data/ember2018/ember_model_2018.txt")
putty_data = open("~/putty.exe", "rb").read()
print(ember.predict_sample(lgbm_model, putty_data))

### **Manual Data**

In [3]:
import codecs
pe = PEFeatureExtractor()
f = utils.ROOT_PATH + "/test/putty.exe"
f = open(f, 'rb')
feat = pe.feature_vector(f.read())
print(feat.shape)

(2381,)


In [12]:
def file2vec(pe, file_path):
    f = open(file_path, "rb")
    feat = pe.feature_vector(f.read())
    feat = feat.reshape(-1, 1)
    return feat 

X = file2vec(pe, utils.ROOT_PATH + "/test/putty.exe")
print(X.shape)

(2381, 1)


### **Testing**

In [5]:
model = lgb.Booster(model_file=utils.ROOT_PATH + "/weights/lightgbm.model")
with open(utils.ROOT_PATH + '/weights/lgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [7]:
model.predict(feat.reshape(1, -1))

array([0.00500452])